In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd


def load_files_from_dir(path):
    data = {}
    list_files = list(path.glob('*.npz'))
    for p in tqdm(list_files):
        try:
            d = np.load(p)
        except:
            continue
        for k, v in d.items():
            data.setdefault(k, []).extend( v)
    indices = np.argsort(data['time_ns'])
    data = {k:np.stack(v)[indices] for k, v in data.items()}
    return data


path_to_files = Path('/mnt/e/Downloads/meas/56625508a1bc4ac4b7ef6dd22ab75807/')

Load data

In [ ]:
pan = load_files_from_dir(path_to_files / 'pan')
mono = load_files_from_dir(path_to_files / 'mono')

Find the closest neighbors between pan and mono (in terms of time_ns)

In [ ]:
# Find the first common index
if len(pan['time_ns']) < len(mono['time_ns']):
    idx = np.searchsorted(mono['time_ns'], pan['time_ns'][0])
    for k in mono.keys():
        mono[k] = mono[k][idx:]
else:
    idx = np.searchsorted(pan['time_ns'], mono['time_ns'][0])
    for k in pan.keys():
        pan[k] = pan[k][idx:]

plt.figure()
plt.plot( pan['time_ns'], label='pan')
plt.plot( mono['time_ns'], label='mono')
plt.legend()
plt.show()


In [ ]:
diff = np.abs(pan['time_ns'][:, None] - mono['time_ns'][None, :])


diff = np.argmin(diff, axis=0)
plt.plot(diff)





# diff = np.min(diff, axis=0)
# diff = diff / 1e9
# # diff = [(pan['time_ns'][p] - q) / 1e9 for p, q in zip(diff, mono['time_ns'])]
# plt.plot(diff[diff < 0.1])

# # [p for p in (diff / 1e9)]
# # 
# # for i in range(len(pan['time_ns'])):
# #     dict_of_connections.setdefault(pan['time_ns'][i], []).append(mono['time_ns'][i])

Build a dict with all the data sorted by the time_ns indices

In [ ]:
rate_mono = len(mono['time_ns']) / ((mono['time_ns'][-1] - mono['time_ns'][0]) / 1e9)
rate_pan = len(pan['time_ns']) / ((pan['time_ns'][-1] - pan['time_ns'][0]) / 1e9)
print(f'rate_mono: {rate_mono}')
print(f'rate_pan: {rate_pan}')

In [ ]:
plt.figure()
plt.plot((pan['time_ns'] - pan['time_ns'][0]) / 1e9, pan['fpa'], label='pan')
plt.plot((mono['time_ns'] - mono['time_ns'][0]) / 1e9, mono['fpa'], label='mono')
plt.legend()
plt.show()


To Dataframe

In [ ]:
images = np.split(pan['frames'], indices_or_sections=len(pan['frames']))
pd.DataFrame(data=[p.squeeze() for p in images], index=pan['time_ns']).head()
# df_pan = pd.DataFrame(pan['fpa'], index=pan['time_ns']).sort_index()
# df_mono = pd.DataFrame(mono['fpa'], index=mono['time_ns']).sort_index()

In [ ]:
df_mono.head()